## 0. 사전 준비

### AWS CLI 설치

In [ ]:
!pip install awscli --quiet

In [ ]:
!aws --version

### 사용자 자격 증명

In [ ]:
import os

# IAM 사용자 등록 필수
os.environ['AWS_ACCESS_KEY_ID'] = '' # TODO: Access Key를 넣으세요
os.environ['AWS_SECRET_ACCESS_KEY'] = '' # TODO: Secret Key를 넣으세요
os.environ['AWS_DEFAULT_REGION'] = 'ap-northeast-2'

# 자격 증명 확인
!aws sts get-caller-identity

## 1. EC2(서버) 인스턴스 생성

### 현재 EC2 목록 출력

In [ ]:
!aws ec2 describe-instances --output table

### 접속을 위한 Key Pair 생성

In [ ]:
# Key Pair 생성
!aws ec2 create-key-pair --key-name my-ec2-key --query KeyMaterial --output text > my-ec2-key.pem

In [ ]:
# Key Pair 생성 확인
!aws ec2 describe-key-pairs --key-names my-ec2-key

### 보안 그룹 생성

In [ ]:
# 보안 그룹 생성
!aws ec2 create-security-group --group-name ec2-sg --description "My security group for EC2"

In [ ]:
# 인바운드 규칙 생성

# local -> ssh 허용
local_ip = '' # TODO : 로컬 PC의 public ip 입력

# SSH 허용 (로컬 -> EC2)
!aws ec2 authorize-security-group-ingress --group-name ec2-sg --protocol tcp --port 22 --cidr {local_ip}/32

# HTTPS 허용 (로컬 -> EC2)
!aws ec2 authorize-security-group-ingress --group-name ec2-sg --protocol tcp --port 443 --cidr {local_ip}/32

In [ ]:
# 현재 보안 그룹 규칙 확인
!aws ec2 describe-security-groups --group-names ec2-sg --query "SecurityGroups[0].IpPermissions" --output table

### EC2 인스턴스 생성

In [ ]:
# EC2 인스턴스 생성
!aws ec2 run-instances \
    --image-id ami-0cf1ead55e8259a57 \
    --instance-type t3.micro \
    --key-name my-ec2-key \
    --security-groups ec2-sg

In [ ]:
# EC2 인스턴스 확인
!aws ec2 describe-instances --query "Reservations[*].Instances[*].[InstanceId,State.Name]" --output table

In [ ]:
# EC2 Instance ID
instance_id = "" # TODO : 위 셀의 실행 결과에서 instance id를 찾아서 넣으세요

### EC2 인스턴스 시작

In [ ]:
!aws ec2 start-instances --instance-ids {instance_id}

### EC2 인스턴스 중지

In [ ]:
!aws ec2 stop-instances --instance-ids {instance_id}

### EC2 인스턴스 종료 (삭제)

In [ ]:
!aws ec2 terminate-instances --instance-ids {instance_id}

## 2. MySQL DB 인스턴스 생성

### 현재 DB 목록 확인

In [ ]:
!aws rds describe-db-instances --output table

In [ ]:
# 기본 VPC의 서브넷 확인
!aws ec2 describe-subnets --query "Subnets[*].[SubnetId,AvailabilityZone]" --output table

### DB 보안 그룹 생성

In [ ]:
# MySQL용 보안 그룹 생성
!aws ec2 create-security-group --group-name mysql-sg --description "MySQL RDS security group"

In [ ]:
# DB Security Group ID 
sg_id = "" # TODO : 위 셀에서 출력된 GroupId 값을 넣으세요

In [ ]:
# MySQL 포트 3306 허용 (EC2 -> DB, 로컬 -> DB)
!aws ec2 authorize-security-group-ingress --group-name mysql-sg --protocol tcp --port 3306 --source-group ec2-sg
!aws ec2 authorize-security-group-ingress --group-name mysql-sg --protocol tcp --port 3306 --cidr {local_ip}/32

### DB 인스턴스 생성

In [ ]:
# MySQL RDS 인스턴스 생성
username = "" # TODO : 원하는 DB username을 넣으세요
password = "" # TODO : 원하는 DB password를 넣으세요

!aws rds create-db-instance \
    --db-name rcmd_practice \
    --db-instance-identifier my-mysql-db \
    --db-instance-class db.t3.micro \
    --engine mysql \
    --engine-version 8.0.42 \
    --master-username {username} \
    --master-user-password {password} \
    --allocated-storage 20 \
    --vpc-security-group-ids {sg_id} \
    --storage-encrypted

In [ ]:
# DB 연결 정보 확인
print("DB Endpoint (None이라면 아직 생성 중인 상태입니다. 생성까지 5분 정도 소요됩니다.)")
!aws rds describe-db-instances --db-instance-identifier my-mysql-db --query "DBInstances[0].Endpoint.Address" --output text

### DB 인스턴스 중지

In [ ]:
# DB 인스턴스 중지
!aws rds stop-db-instance --db-instance-identifier my-mysql-db

### DB 인스턴스 시작

In [ ]:
# DB 인스턴스 시작
!aws rds start-db-instance --db-instance-identifier my-mysql-db

### DB 인스턴스 삭제

In [ ]:
# DB 인스턴스 삭제
!aws rds delete-db-instance --db-instance-identifier my-mysql-db --skip-final-snapshot

### 라이브러리 설치

In [ ]:
!pip install pymysql --quiet

### DB 연결

In [ ]:
import pymysql

db_endpoint = "" # TODO

DB_CONFIG = {
    'host': db_endpoint,
    'user': username,
    'password': password,
    'database':'rcmd_practice',
    'charset': 'utf8mb4',
    "client_flag": pymysql.constants.CLIENT.MULTI_STATEMENTS,
    'cursorclass': pymysql.cursors.DictCursor
}

connection = pymysql.connect(**DB_CONFIG)

### DB 스크립트 실행 (테이블 생성, 데이터 삽입)

In [ ]:
with open("db_init.sql", 'r') as file:
    query = file.read().replace("\n", " ")

connection.query(query)
connection.commit()

### 데이터 확인 및 연결 종료

In [ ]:
SAMPLE_SIZE = 5

try:
    with connection.cursor() as cursor:

        tables = ['movies', 'users', 'rcmd_user_movie']
        
        for table in tables:
            print(f"\n--- Data in {table} ---")
            cursor.execute(f"SELECT * FROM `{table}`;")
            rows = cursor.fetchmany(SAMPLE_SIZE)
            
            if rows:
                for row in rows:
                    print(row)
            else:
                print("(No data found)")

finally:
    connection.close()